In [1]:
import pandas as pd
import numpy as np
from os import listdir
import keras
# import cv2
from skimage.io import imread
from skimage.transform import resize
import tensorflow as tf

Using TensorFlow backend.


In [2]:
train_path = "/projectnb/dl-course/jxtang/EC500_proj/train/"
test_path = "/projectnb/dl-course/jxtang/EC500_proj/test/"
label_path = './train.csv'
train_files = listdir(train_path)
test_files = listdir(test_path)


In [3]:
train_labels = pd.read_csv(label_path)
train_labels.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [4]:
names_dict = {
    0:  "Nucleoplasm",  
    1:  "Nuclear membrane",   
    2:  "Nucleoli",   
    3:  "Nucleoli fibrillar center",   
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",   
    6:  "Endoplasmic reticulum",   
    7:  "Golgi apparatus",   
    8:  "Peroxisomes",   
    9:  "Endosomes",   
    10:  "Lysosomes",   
    11:  "Intermediate filaments",   
    12:  "Actin filaments",   
    13:  "Focal adhesion sites",   
    14:  "Microtubules",   
    15:  "Microtubule ends",   
    16:  "Cytokinetic bridge",   
    17:  "Mitotic spindle",   
    18:  "Microtubule organizing center",   
    19:  "Centrosome",   
    20:  "Lipid droplets",   
    21:  "Plasma membrane",   
    22:  "Cell junctions",   
    23:  "Mitochondria",   
    24:  "Aggresome",   
    25:  "Cytosol",   
    26:  "Cytoplasmic bodies",   
    27:  "Rods & rings"
}
# reverse_names_dict = dict((v,k) for k,v in names_dict.items())

In [5]:
def fill_targets(row):
    row.Target = np.array(row.Target.split(" ")).astype(np.int)
    for num in row.Target:
        name = names_dict[int(num)]
        row.loc[name] = 1
    return row

for key in names_dict.keys():
    train_labels[names_dict[key]] = 0
train_labels = train_labels.apply(fill_targets, axis=1)
train_labels["number_of_targets"] = train_labels.drop(["Id", "Target"],axis=1).sum(axis=1)

In [6]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
for train_idx, test_idx in kf.split(train_labels.index.values):
    partition = {}
    partition["train"] = train_labels.Id.values[train_idx]
    partition["validation"] = train_labels.Id.values[test_idx]
#     X_train, X_test = train_labels.Id.values[train_idx], train_labels.Id.values[test_idx]
#     y_train, y_test = train_labels.Id.values[train_idx], train_labels.Id.values[test_idx]

In [7]:
partition

{'train': array(['00070df0-bbc3-11e8-b2bc-ac1f6b6435d0',
        '000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0',
        '000a9596-bbc4-11e8-b2bc-ac1f6b6435d0', ...,
        'ab33dff8-bba7-11e8-b2ba-ac1f6b6435d0',
        'ab351f1c-bbb6-11e8-b2ba-ac1f6b6435d0',
        'ab385b2e-bbba-11e8-b2ba-ac1f6b6435d0'], dtype=object),
 'validation': array(['ab3978aa-bbbb-11e8-b2ba-ac1f6b6435d0',
        'ab3b9258-bbab-11e8-b2ba-ac1f6b6435d0',
        'ab3ca16e-bbc1-11e8-b2bb-ac1f6b6435d0', ...,
        'fff189d8-bbab-11e8-b2ba-ac1f6b6435d0',
        'fffdf7e0-bbc4-11e8-b2bc-ac1f6b6435d0',
        'fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0'], dtype=object)}

In [8]:
df = pd.read_csv('sample_submission.csv')

In [11]:
partition2 = np.array(df['Id'])
partition2

array(['00008af0-bad0-11e8-b2b8-ac1f6b6435d0',
       '0000a892-bacf-11e8-b2b8-ac1f6b6435d0',
       '0006faa6-bac7-11e8-b2b7-ac1f6b6435d0', ...,
       'ffecb8a4-bad4-11e8-b2b8-ac1f6b6435d0',
       'fff03816-bad5-11e8-b2b9-ac1f6b6435d0',
       'fffe6f9c-bacd-11e8-b2b8-ac1f6b6435d0'], dtype=object)

In [24]:
class ModelParameters(object):
    path = train_path
    num_classes=28
    image_rows=512
    image_cols=512
    batch_size=50
    n_channels=3
    shuffle=False
    scaled_row_dim = 197
    scaled_col_dim = 197 
    n_epochs=300

parameter = ModelParameters()

In [9]:
class ImagePreprocessor:
    
    def __init__(self, modelparameter):
        self.parameter = modelparameter
        self.path = self.parameter.path
        self.scaled_row_dim = self.parameter.scaled_row_dim
        self.scaled_col_dim = self.parameter.scaled_col_dim
        self.n_channels = self.parameter.n_channels
    
    def preprocess(self, image):
        image = self.resize(image)
        image = self.reshape(image)
        image = self.normalize(image)
        return image
    
    def resize(self, image):
        return resize(image, (self.scaled_row_dim, self.scaled_col_dim))
    
    def reshape(self, image):
        return np.reshape(image, (image.shape[0], image.shape[1], self.n_channels))
    
    def normalize(self, image):
#         image /= 255
#         return image
        return (image / 255.0 - 0.5) / 0.5
            
    def load_image(self, image_id):
        image = np.zeros(shape=(512,512,4))
        image[:,:,0] = imread(self.basepath + image_id + "_green" + ".png")
        image[:,:,1] = imread(self.basepath + image_id + "_blue" + ".png")
        image[:,:,2] = imread(self.basepath + image_id + "_red" + ".png")
        image[:,:,3] = imread(self.basepath + image_id + "_yellow" + ".png")
        return image[:,:,0:self.parameter.n_channels]

In [10]:
preprocessor = ImagePreprocessor(parameter)

In [11]:
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, list_IDs, labels, modelparameter, imagepreprocessor):
        self.params = modelparameter
        self.labels = labels
        self.list_IDs = list_IDs
        self.dim = (self.params.scaled_row_dim, self.params.scaled_col_dim)
        self.batch_size = self.params.batch_size
        self.n_channels = self.params.n_channels
        self.num_classes = self.params.num_classes
        self.preprocessor = imagepreprocessor
        self.shuffle = self.params.shuffle
        self.on_epoch_end()
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def get_targets_per_image(self, identifier):
        return self.labels.loc[self.labels.Id==identifier].drop(
                ["Id", "Target", "number_of_targets"], axis=1).values
            
    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.num_classes), dtype=int)
        for i, identifier in enumerate(list_IDs_temp):
            image = self.preprocessor.load_image(identifier)
            image = self.preprocessor.preprocess(image)
            X[i] = image
            y[i] = self.get_targets_per_image(identifier)
        return X, y
    
    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
 
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

In [12]:
class PredictGenerator:
    
    def __init__(self, predict_Ids, imagepreprocessor, predict_path):
        self.preprocessor = imagepreprocessor
        self.preprocessor.basepath = predict_path
        self.identifiers = predict_Ids
    
    def predict(self, model):
        y = np.empty(shape=(len(self.identifiers), self.preprocessor.parameter.num_classes))
        for n in range(len(self.identifiers)):
            image = self.preprocessor.load_image(self.identifiers[n])
            image = self.preprocessor.preprocess(image)
            image = image.reshape((1, *image.shape))
            y[n] = model.predict(image)
        return y

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.losses import binary_crossentropy
from keras.optimizers import Adadelta
from keras.models import load_model

class BaseLineModel:
    
    def __init__(self, modelparameter):
        self.params = modelparameter
        self.num_classes = self.params.num_classes
        self.img_rows = self.params.scaled_row_dim
        self.img_cols = self.params.scaled_col_dim
        self.n_channels = self.params.n_channels
        self.input_shape = (self.img_rows, self.img_cols, self.n_channels)
        self.my_metrics = ['accuracy']
    
    def build_model(self):
        self.model = Sequential()
        self.model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=self.input_shape))
        self.model.add(Conv2D(32, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        self.model.add(Flatten())
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(self.num_classes, activation='sigmoid'))

    def compile_model(self):
        self.model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=self.my_metrics)
    
    def set_generators(self, train_generator, validation_generator):
        self.training_generator = train_generator
        self.validation_generator = validation_generator
    
    def learn(self):
        return self.model.fit_generator(generator=self.training_generator,
                    validation_data=self.validation_generator,
                    epochs=self.params.n_epochs, 
                    steps_per_epoch=100,
                    use_multiprocessing=True,
                    validation_steps=50,
                    workers=8)
    
    def score(self):
        return self.model.evaluate_generator(generator=self.validation_generator,
                                      use_multiprocessing=True, 
                                      workers=8)

    def predict(self, predict_generator):
        y = predict_generator.predict(self.model)
        return y
    
    def save(self, modeloutputpath):
        self.model.save(modeloutputpath)
    
    def load(self, modelinputpath):
        self.model = load_model(modelinputpath)

In [14]:
labels = train_labels
print("Number of samples in train: {}".format(len(partition["train"])))
print("Number of samples in validation: {}".format(len(partition["validation"])))

Number of samples in train: 20715
Number of samples in validation: 10357


In [15]:
training_generator = DataGenerator(partition['train'], labels, parameter, preprocessor)
validation_generator = DataGenerator(partition['validation'], labels, parameter, preprocessor)
predict_generator = PredictGenerator(partition2, preprocessor, test_path)

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10414248683350009237
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11797744845
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 3863235766506655027
physical_device_desc: "device: 0, name: Tesla P100-PCIE-12GB, pci bus id: 0000:03:00.0, compute capability: 6.0"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 11797744845
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 5083360453508409716
physical_device_desc: "device: 1, name: Tesla P100-PCIE-12GB, pci bus id: 0000:04:00.0, compute capability: 6.0"
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 11797744845
locality {
  bus_id: 2
  numa_node: 1
  links {
    link {
      device_id: 3
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 5088028355621

In [17]:
import keras.backend as K

def base_f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return f1

def f1_min(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.min(f1)

def f1_max(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.max(f1)

def f1_mean(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.mean(f1)

def f1_std(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.std(f1)

In [18]:
class TrackHistory(keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [19]:
class ImprovedDataGenerator(DataGenerator):
    
    # in contrast to the base DataGenerator we add a target wishlist to init
    def __init__(self, list_IDs, labels, modelparameter, imagepreprocessor, target_wishlist):
        super().__init__(list_IDs, labels, modelparameter, imagepreprocessor)
        self.target_wishlist = target_wishlist
    
    def get_targets_per_image(self, identifier):
        return self.labels.loc[self.labels.Id==identifier][self.target_wishlist].values


In [25]:
use_dropout = False
from keras.models import Model, Sequential,load_model
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.losses import binary_crossentropy
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adadelta,Adam
from keras.models import load_model
import tensorflow as tf
from keras.layers import Input

class ImprovedModel(BaseLineModel):
    
    def __init__(self, modelparameter,
                 use_dropout,
                 my_metrics=[f1_mean, f1_std, f1_min, f1_max]):
        
        super().__init__(modelparameter)
        self.my_metrics = my_metrics
        self.use_dropout = use_dropout
        
    def learn(self):
        self.history = TrackHistory()
        return self.model.fit_generator(generator=self.training_generator,
                    validation_data=self.validation_generator,
                    epochs=self.params.n_epochs, 
                    use_multiprocessing=True,
                    workers=8,
                    steps_per_epoch=100,
                    validation_steps=50,                                     
                    callbacks = [self.history])
    
    def build_model(self):
        #input_tensor = Input(shape=self.input_shape)
        pretrain_model = ResNet50(
            include_top=True, 
            #weights='imagenet', 
            weights = None,
            input_shape=self.input_shape,
            #input_tensor = input_tensor
            classes = 28
            ) 
        
        self.model = pretrain_model
        
#         self.model = Sequential()
# #         for l in pretrain_model.layers:
# #             self.model.add(l)            
#         self.model.add(pretrain_model)
#         self.model.add(Conv2D(128, kernel_size=(1,1), activation='relu'))
#         self.model.add(Flatten())
#         if self.use_dropout:
#             self.model.add(Dropout(0.25))
#         self.model,add(Dense(512, activation='relu'))
#         if self.use_dropout:
#             self.model.add(Dropout(0.25))
#         self.model.add(Dense(self.num_classes, activation='sigmoid'))         
    

In [26]:
config = tf.ConfigProto(intra_op_parallelism_threads=4,\
        inter_op_parallelism_threads=4, allow_soft_placement=True,\
        device_count = {'CPU' : 4, 'GPU' : 1})
session = tf.Session(config=config)
K.set_session(session)

In [27]:
parameter = ModelParameters()
preprocessor = ImagePreprocessor(parameter)
labels = train_labels
training_generator = DataGenerator(partition['train'], labels,
                                           parameter, preprocessor)
validation_generator = DataGenerator(partition['validation'], labels,
                                             parameter, preprocessor)
predict_generator = PredictGenerator(partition['validation'], preprocessor, train_path)

In [23]:
import warnings
warnings.filterwarnings("ignore")
model = ImprovedModel(parameter, use_dropout=use_dropout)
model.build_model()
model.compile_model()
model.set_generators(training_generator, validation_generator)
epoch_history = model.learn()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/150
100/100 [==============================] - 97s - loss: 0.2087 - f1_mean: 0.0039 - f1_std: 0.0196 - f1_min: 0.0000e+00 - f1_max: 0.1037 - val_loss: 0.2078 - val_f1_mean: 0.0000e+00 - val_f1_std: 0.0000e+00 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0000e+00
Epoch 2/150
100/100 [==============================] - 63s - loss: 0.1934 - f1_mean: 0.0069 - f1_std: 0.0354 - f1_min: 0.0000e+00 - f1_max: 0.1893 - val_loss: 0.2029 - val_f1_mean: 5.5050e-04 - val_f1_std: 0.0029 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0154
Epoch 3/150
100/100 [==============================] - 61s - loss: 0.1905 - f1_mean: 0.0076 - f1_std: 0.0380 

100/100 [==============================] - 63s - loss: 0.1312 - f1_mean: 0.0826 - f1_std: 0.2191 - f1_min: 0.0000e+00 - f1_max: 0.8464 - val_loss: 0.1954 - val_f1_mean: 0.0453 - val_f1_std: 0.1723 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7989
Epoch 59/150
100/100 [==============================] - 63s - loss: 0.1359 - f1_mean: 0.0819 - f1_std: 0.2156 - f1_min: 0.0000e+00 - f1_max: 0.8346 - val_loss: 0.1790 - val_f1_mean: 0.0543 - val_f1_std: 0.1748 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7403
Epoch 60/150
100/100 [==============================] - 60s - loss: 0.1352 - f1_mean: 0.0786 - f1_std: 0.2123 - f1_min: 0.0000e+00 - f1_max: 0.8332 - val_loss: 0.1953 - val_f1_mean: 0.0380 - val_f1_std: 0.1416 - val_f1_min: 0.0000e+00 - val_f1_max: 0.6557
Epoch 61/150
100/100 [==============================] - 63s - loss: 0.1332 - f1_mean: 0.0789 - f1_std: 0.2154 - f1_min: 0.0000e+00 - f1_max: 0.8414 - val_loss: 0.2026 - val_f1_mean: 0.0616 - val_f1_std: 0.1867 - val_f1_min: 0.0000e+00 - val_f1_m

100/100 [==============================] - 63s - loss: 0.0785 - f1_mean: 0.2410 - f1_std: 0.3767 - f1_min: 0.0000e+00 - f1_max: 0.9967 - val_loss: 0.3402 - val_f1_mean: 0.0270 - val_f1_std: 0.0927 - val_f1_min: 0.0000e+00 - val_f1_max: 0.4196
Epoch 122/150
100/100 [==============================] - 62s - loss: 0.0827 - f1_mean: 0.2302 - f1_std: 0.3719 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.2249 - val_f1_mean: 0.0871 - val_f1_std: 0.2148 - val_f1_min: 0.0000e+00 - val_f1_max: 0.8209
Epoch 123/150
100/100 [==============================] - 62s - loss: 0.0817 - f1_mean: 0.2362 - f1_std: 0.3714 - f1_min: 0.0000e+00 - f1_max: 0.9887 - val_loss: 0.2221 - val_f1_mean: 0.0572 - val_f1_std: 0.1735 - val_f1_min: 0.0000e+00 - val_f1_max: 0.6991
Epoch 124/150
100/100 [==============================] - 64s - loss: 0.0821 - f1_mean: 0.2321 - f1_std: 0.3732 - f1_min: 0.0000e+00 - f1_max: 0.9983 - val_loss: 0.2497 - val_f1_mean: 0.0543 - val_f1_std: 0.1640 - val_f1_min: 0.0000e+00 - val_f

In [28]:
import warnings
warnings.filterwarnings("ignore")
model = ImprovedModel(parameter, use_dropout=use_dropout)
model.build_model()
model.compile_model()
model.set_generators(training_generator, validation_generator)
epoch_history = model.learn()

Epoch 1/300
100/100 [==============================] - 149s - loss: 0.2035 - f1_mean: 0.0053 - f1_std: 0.0263 - f1_min: 0.0000e+00 - f1_max: 0.1392 - val_loss: 0.2048 - val_f1_mean: 0.0000e+00 - val_f1_std: 0.0000e+00 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0000e+00
Epoch 2/300
100/100 [==============================] - 104s - loss: 0.1830 - f1_mean: 0.0068 - f1_std: 0.0325 - f1_min: 0.0000e+00 - f1_max: 0.1687 - val_loss: 0.1972 - val_f1_mean: 0.0000e+00 - val_f1_std: 0.0000e+00 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0000e+00
Epoch 3/300
100/100 [==============================] - 104s - loss: 0.1824 - f1_mean: 0.0111 - f1_std: 0.0525 - f1_min: 0.0000e+00 - f1_max: 0.2745 - val_loss: 0.1947 - val_f1_mean: 7.5188e-05 - val_f1_std: 3.9069e-04 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0021
Epoch 4/300
100/100 [==============================] - 102s - loss: 0.1814 - f1_mean: 0.0127 - f1_std: 0.0614 - f1_min: 0.0000e+00 - f1_max: 0.3234 - val_loss: 0.1919 - val_f1_mean: 0.0133 - val_f1_st

100/100 [==============================] - 105s - loss: 0.0711 - f1_mean: 0.4023 - f1_std: 0.4094 - f1_min: 0.0000e+00 - f1_max: 0.9987 - val_loss: 0.2958 - val_f1_mean: 0.0378 - val_f1_std: 0.1116 - val_f1_min: 0.0000e+00 - val_f1_max: 0.4839
Epoch 64/300
100/100 [==============================] - 103s - loss: 0.0642 - f1_mean: 0.4145 - f1_std: 0.4155 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.3108 - val_f1_mean: 0.0606 - val_f1_std: 0.1524 - val_f1_min: 0.0000e+00 - val_f1_max: 0.6136
Epoch 65/300
100/100 [==============================] - 103s - loss: 0.0636 - f1_mean: 0.4113 - f1_std: 0.4145 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.3121 - val_f1_mean: 0.0345 - val_f1_std: 0.0984 - val_f1_min: 0.0000e+00 - val_f1_max: 0.4281
Epoch 66/300
100/100 [==============================] - 104s - loss: 0.0646 - f1_mean: 0.4079 - f1_std: 0.4126 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.2709 - val_f1_mean: 0.0663 - val_f1_std: 0.1572 - val_f1_min: 0.0000e+00 - val_

100/100 [==============================] - 105s - loss: 0.0409 - f1_mean: 0.4380 - f1_std: 0.4245 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.2869 - val_f1_mean: 0.0771 - val_f1_std: 0.1874 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7330
Epoch 126/300
100/100 [==============================] - 103s - loss: 0.0408 - f1_mean: 0.4322 - f1_std: 0.4215 - f1_min: 0.0000e+00 - f1_max: 0.9994 - val_loss: 0.2499 - val_f1_mean: 0.1074 - val_f1_std: 0.2132 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7658
Epoch 127/300
100/100 [==============================] - 104s - loss: 0.0396 - f1_mean: 0.4201 - f1_std: 0.4214 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.2378 - val_f1_mean: 0.1075 - val_f1_std: 0.2212 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7698
Epoch 128/300
100/100 [==============================] - 104s - loss: 0.0403 - f1_mean: 0.4258 - f1_std: 0.4208 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.3278 - val_f1_mean: 0.0458 - val_f1_std: 0.1323 - val_f1_min: 0.0000e+00 - v

100/100 [==============================] - 105s - loss: 0.0353 - f1_mean: 0.4182 - f1_std: 0.4205 - f1_min: 0.0000e+00 - f1_max: 0.9987 - val_loss: 0.3134 - val_f1_mean: 0.1040 - val_f1_std: 0.2133 - val_f1_min: 0.0000e+00 - val_f1_max: 0.8108
Epoch 188/300
100/100 [==============================] - 104s - loss: 0.0354 - f1_mean: 0.4247 - f1_std: 0.4220 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.2359 - val_f1_mean: 0.1222 - val_f1_std: 0.2279 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7585
Epoch 189/300
100/100 [==============================] - 105s - loss: 0.0356 - f1_mean: 0.4235 - f1_std: 0.4235 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.2616 - val_f1_mean: 0.1298 - val_f1_std: 0.2411 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7994
Epoch 190/300
100/100 [==============================] - 104s - loss: 0.0359 - f1_mean: 0.4180 - f1_std: 0.4184 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.2725 - val_f1_mean: 0.1009 - val_f1_std: 0.2124 - val_f1_min: 0.0000e+00 - v

100/100 [==============================] - 102s - loss: 0.0347 - f1_mean: 0.4238 - f1_std: 0.4205 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.3886 - val_f1_mean: 0.1039 - val_f1_std: 0.2083 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7536
Epoch 250/300
100/100 [==============================] - 103s - loss: 0.0351 - f1_mean: 0.4214 - f1_std: 0.4182 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.2930 - val_f1_mean: 0.1097 - val_f1_std: 0.2137 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7497
Epoch 251/300
100/100 [==============================] - 104s - loss: 0.0353 - f1_mean: 0.4150 - f1_std: 0.4184 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.2484 - val_f1_mean: 0.1294 - val_f1_std: 0.2437 - val_f1_min: 0.0000e+00 - val_f1_max: 0.8346
Epoch 252/300
100/100 [==============================] - 124s - loss: 0.0343 - f1_mean: 0.4180 - f1_std: 0.4197 - f1_min: 0.0000e+00 - f1_max: 0.9980 - val_loss: 0.2496 - val_f1_mean: 0.0996 - val_f1_std: 0.2085 - val_f1_min: 0.0000e+00 - v

In [ ]:
proba_predictions = model.predict(predict_generator)

In [ ]:
improved_proba_predictions = pd.DataFrame(proba_predictions)
improved_proba_predictions.to_excel('resnet50_result.xlsx')

In [ ]:
import warnings
warnings.filterwarnings("ignore")
model = ImprovedModel(parameter, use_dropout=use_dropout)
model.build_model()
model.compile_model()
model.set_generators(training_generator, validation_generator)
epoch_history = model.learn()
proba_predictions = model.predict(predict_generator)

Epoch 1/100
100/100 [==============================] - 269s - loss: 0.1916 - f1_mean: 0.0027 - f1_std: 0.0140 - f1_min: 0.0000e+00 - f1_max: 0.0746 - val_loss: 0.1993 - val_f1_mean: 0.0000e+00 - val_f1_std: 0.0000e+00 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0000e+00
Epoch 2/100
100/100 [==============================] - 230s - loss: 0.1859 - f1_mean: 0.0060 - f1_std: 0.0299 - f1_min: 0.0000e+00 - f1_max: 0.1592 - val_loss: 0.2015 - val_f1_mean: 0.0066 - val_f1_std: 0.0341 - val_f1_min: 0.0000e+00 - val_f1_max: 0.1836
Epoch 3/100
100/100 [==============================] - 231s - loss: 0.1830 - f1_mean: 0.0097 - f1_std: 0.0465 - f1_min: 0.0000e+00 - f1_max: 0.2428 - val_loss: 0.1913 - val_f1_mean: 7.1429e-05 - val_f1_std: 3.7115e-04 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0020
Epoch 4/100
100/100 [==============================] - 231s - loss: 0.1838 - f1_mean: 0.0119 - f1_std: 0.0561 - f1_min: 0.0000e+00 - f1_max: 0.2943 - val_loss: 0.1865 - val_f1_mean: 0.0063 - val_f1_std: 0.0316 - 

In [38]:
#model.save("improved_model.h5")
improved_proba_predictions = pd.DataFrame(proba_predictions)
pd.DataFrame(improved_proba_predictions)

KeyboardInterrupt: 